In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os.path
import itertools

The BBL number is going to be ideal for indexing as it is used for this purpose by the NYC databases. https://www.pincusco.com/data-dictionary/bbl-also-known-as-borough-block-and-lot-or-property-id/

In [ ]:
# Add the BBL column for indexing
data['bbl'] = data['BORO'].astype(str) + data['BLOCK'].astype(str) + data['LOT'].astype(str)
# Add the BBL column for indexing
clean['bbl'] = clean['borough'].astype(str) + clean['block'].astype(str) + clean['lot'].astype(str)